# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = "../output_data/city_data_df.csv"
city_data_df =pd.read_csv(csv_file)
city_data_df.head()


City Country      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0     Busselton      AU -33.6500  115.3333     68.00        70          32   
1        Barrow      US  71.2906 -156.7887     -5.80        84           1   
2      Khatanga      RU  71.9667  102.5000    -19.19        87         100   
3  Punta Arenas      CL -53.1500  -70.9167     46.40        49          20   
4       Caucaia      BR  -3.7361  -38.6531     81.00        83          40   

   Wind Speed        Date  
0        3.00  1611619906  
1       12.66  1611619802  
2        8.70  1611619906  
3       29.93  1611619906  
4        6.91  1611619846

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Create a list containing coordinates
corrdinates = city_data_df[['Lat','Lng']]
#print(corrdinates)

In [5]:
# create humidity list
humidity = city_data_df['Humidity'].astype(int)
#print(humidity)

In [7]:
# plot map https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#heatmaps

# just the map figure

fig = gmaps.figure()
# creates layer and locations
heat_layer = gmaps.heatmap_layer(corrdinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2 )
# # add heat layer over base layer
fig.add_layer(heat_layer)

#display all layers
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

  Idea Conditions:
  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

In [8]:
# set idea conditions with multiconditional statement 
# https://www.kite.com/python/answers/how-to-select-rows-by-multiple-label-conditions-with-pandas-loc-in-python
idea_conditions_df = city_data_df.loc[(city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 80) &
                                     (city_data_df['Wind Speed'] <= 10) &
                                     (city_data_df['Cloudiness'] ==0)]

idea_conditions_df.head()

City Country      Lat      Lng  Max Temp  Humidity  Cloudiness  \
81     Saint-Joseph      RE -21.3667  55.6167     77.00        73           0   
86   Saint-Philippe      RE -21.3585  55.7679     77.00        73           0   
130     Tambacounda      SN  13.7539 -13.7586     73.40        20           0   
322           Tabou      CI   4.4230  -7.3528     77.92        87           0   
338           Podor      MR  16.6527 -14.9587     73.40        25           0   

     Wind Speed        Date  
81         6.91  1611619916  
86         6.91  1611619916  
130        9.22  1611619921  
322        7.23  1611619942  
338        2.30  1611619944

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#add Hotel Name Column to DF
idea_conditions_df['Hotel Name'] = ""
idea_conditions_df.head()

C:\Users\Chad\anaconda\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


City Country      Lat      Lng  Max Temp  Humidity  Cloudiness  \
81     Saint-Joseph      RE -21.3667  55.6167     77.00        73           0   
86   Saint-Philippe      RE -21.3585  55.7679     77.00        73           0   
130     Tambacounda      SN  13.7539 -13.7586     73.40        20           0   
322           Tabou      CI   4.4230  -7.3528     77.92        87           0   
338           Podor      MR  16.6527 -14.9587     73.40        25           0   

     Wind Speed        Date Hotel Name  
81         6.91  1611619916             
86         6.91  1611619916             
130        9.22  1611619921             
322        7.23  1611619942             
338        2.30  1611619944

In [10]:
cities = idea_conditions_df['City']
print(cities)

81       Saint-Joseph
86     Saint-Philippe
130       Tambacounda
322             Tabou
338             Podor
358           Salalah
381            Harper
430          Moratuwa
471             Dalby
555         Takoradze
Name: City, dtype: object


In [12]:
# variable to store hotel names to add to DF
hotel_df = []
# params found in documentation radius is default meters type is lodging-name the keys what is required by documentation params
params = {'radius' : 5000,
         'type' : "lodging",
         'key' : g_key}



# loop to find hotel info based on idea conditions df
for index, row in idea_conditions_df.iterrows():
    #obtain lat and lng per city
    lat = row['Lat']
    lng = row['Lng']
    # update params add new key for corridinates-name the keys what is required by documentation params
    params['location'] =f"{lat},{lng}"

    #base url to manipulate each itteration of the loop
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #modified url with location info
    response = requests.get(base_url, params = params).json()
    #print dumps for review
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #retrives the results portion of the json dict with hotel name in 0 element
    hotels = response['results']
    
    # try and excpet for errors/print to see loop in action
    try:
        print(f"{hotels[0]['name']} is the closest lodging.")
        #update df with hotel name
        idea_conditions_df.loc[index, 'Hotel Name'] = hotels[0]['name']
    except (KeyError, IndexError):
        print("Error found skipping")
        
    


"Plantation Bed and Breakfast is the closest lodging.


C:\Users\Chad\anaconda\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Chambres d'hôte "La Trinité" is the closest lodging.
Error found skipping
hotêl doufoulougou is the closest lodging.
La cour du fleuve is the closest lodging.
HAMDAN PLAZA HOTEL SALALAH is the closest lodging.
Screensaver is the closest lodging.
Sa Mansion By Yoho is the closest lodging.
Kobbers Motor Inn Dalby is the closest lodging.
Raybow International Hotel is the closest lodging.


In [13]:
hotel_df = idea_conditions_df
hotel_df.head()

City Country      Lat      Lng  Max Temp  Humidity  Cloudiness  \
81     Saint-Joseph      RE -21.3667  55.6167     77.00        73           0   
86   Saint-Philippe      RE -21.3585  55.7679     77.00        73           0   
130     Tambacounda      SN  13.7539 -13.7586     73.40        20           0   
322           Tabou      CI   4.4230  -7.3528     77.92        87           0   
338           Podor      MR  16.6527 -14.9587     73.40        25           0   

     Wind Speed        Date                     Hotel Name  
81         6.91  1611619916  "Plantation Bed and Breakfast  
86         6.91  1611619916   Chambres d'hôte "La Trinité"  
130        9.22  1611619921                                 
322        7.23  1611619942             hotêl doufoulougou  
338        2.30  1611619944              La cour du fleuve

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Create a list containing coordinates
# corrdinates = hotel_df[['Lat','Lng']]
#print(corrdinates)

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations, 
    info_box_content= hotel_info)

# Display figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))